In [ ]:
import pandas as pd

In [ ]:

def merge_stock_and_news_data(company_name):
    company_data_path = f'drive/MyDrive/SoftComp/Dhaka Stock Exchange Historical Data/Adjusted Data/{company_name}.csv'
    news_data_path = 'drive/MyDrive/SoftComp/prothomalo_data.csv'

    # Read data from CSV files
    stock_df = pd.read_csv(company_data_path)
    news_df = pd.read_csv(news_data_path)

    # Columns to select from each DataFrame
    cols_from_stock = ['Date', 'Close']
    cols_from_news = ['Date', 'headline']

    # Convert 'Date' column to datetime format
    stock_df['Date'] = pd.to_datetime(stock_df['Date']).dt.strftime('%d-%m-%Y')
    news_df.rename(columns={'published_at': 'Date'}, inplace=True)

    # Merge DataFrames on 'Date' column
    merged_df = pd.merge(news_df[cols_from_news], stock_df[cols_from_stock], on='Date', how='inner')

    return merged_df

# Example usage:
company_name = "BEXIMCO"

merged_df = merge_stock_and_news_data(company_name)
print(merged_df.head())

         Date                                      headline  Close
0  06-07-2022             জিপি ও রবির দাপটে লেনদেন বাড়ল ৪৫%  132.9
1  30-06-2022  ৬ মাসে বন্ধ কোম্পানির শেয়ারের দাম বেড়ে ৫ গুণ  129.8
2  26-06-2022                      ডিএসইতে লেনদেন কমেছে ১১%  132.2
3  16-06-2022       ‘কালোটাকা বিনিয়োগের সুযোগ বিবেচনাযোগ্য’  134.6
4  15-06-2022                   টানা দরপতন চলছে শেয়ারবাজারে  131.3


# LSTM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Input, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming your dataset is stored in a DataFrame called 'merged_df'
# Extract features and target
X_text = merged_df['headline'].values
X_date = pd.to_datetime(merged_df['Date']).values
X_numerical = merged_df[['Volume', 'Open', 'High', 'Low']].values
y = merged_df['Close'].values

# Text data preprocessing
max_words = 1000  # Set the maximum number of words in your vocabulary
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_text)
X_text_sequence = tokenizer.texts_to_sequences(X_text)
X_text_padded = pad_sequences(X_text_sequence, padding='post')

# Convert datetime to numeric format (e.g., Unix timestamp)
X_date_numeric = X_date.astype(np.int64) // 10**9  # Convert to seconds for simplicity

# Normalize numerical data
scaler = MinMaxScaler()
X_numerical = scaler.fit_transform(X_numerical)

# Define the input shapes for each type of data
text_input_shape = X_text_padded.shape[1]
date_input_shape = 1  # Scalar input for date
numerical_input_shape = X_numerical.shape[1]

# Define input layers
text_input = Input(shape=(text_input_shape,))
date_input = Input(shape=(date_input_shape,))
numerical_input = Input(shape=(numerical_input_shape,))

# Embedding layer for text input
embedded_text = Embedding(input_dim=max_words, output_dim=50, input_length=text_input_shape)(text_input)
lstm_out = LSTM(50)(embedded_text)

# Concatenate the outputs from different input branches
concatenated = concatenate([lstm_out, date_input, numerical_input])

# Dense layer
output = Dense(1)(concatenated)

# Define the model
model = Model(inputs=[text_input, date_input, numerical_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train-test split
X_train_text, X_test_text, X_train_date, X_test_date, X_train_numerical, X_test_numerical, y_train, y_test = train_test_split(
    X_text_padded, X_date_numeric, X_numerical, y, test_size=0.2, random_state=42
)

# Train the model
model.fit(
    [X_train_text, X_train_date, X_train_numerical], y_train, epochs=100, batch_size=32,
    validation_data=([X_test_text, X_test_date, X_test_numerical], y_test)
)

# Evaluate the model
loss = model.evaluate([X_test_text, X_test_date, X_test_numerical], y_test)
print(f'Mean Squared Error on Test Data: {loss}')


<ipython-input-59-861a6962154d>:13: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  X_date = pd.to_datetime(merged_df['Date']).values


KeyError: "None of [Index(['Volume', 'Open', 'High', 'Low'], dtype='object')] are in the [columns]"

# Ensemble (Early Fusion)

In [ ]:
import numpy as np
import pandas as pd
import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Conv1D, Dense, Embedding, Flatten, Input, LSTM, MaxPooling1D, Reshape, ZeroPadding1D, concatenate, RepeatVector
from tensorflow.keras.models import Model
from transformers import AutoModel, AutoTokenizer


In [ ]:
# Load the datasets
# df = pd.read_csv("your_dataset.csv")

# Convert date to datetime format
# df["Date"] = pd.to_datetime(df["Date"], format='%d-%m-%Y')

# Sort by date
# df = df.sort_values("Date")

df = merged_df
# df_test = merged_df.iloc[15:30]
# Split into X (headlines and prices) and y (closing price)
df.head()

,Date,headline,Close
0,06-07-2022,জিপি ও রবির দাপটে লেনদেন বাড়ল ৪৫%,132.9
1,30-06-2022,৬ মাসে বন্ধ কোম্পানির শেয়ারের দাম বেড়ে ৫ গুণ,129.8
2,26-06-2022,ডিএসইতে লেনদেন কমেছে ১১%,132.2
3,16-06-2022,‘কালোটাকা বিনিয়োগের সুযোগ বিবেচনাযোগ্য’,134.6
4,15-06-2022,টানা দরপতন চলছে শেয়ারবাজারে,131.3


In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from transformers import TFBertModel, BertTokenizer

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
bangla_model = TFBertModel.from_pretrained("sagorsarker/bangla-bert-base", from_pt=True)

# Define a function to encode Bengali text
def encode_bengali_text(text):
    input_ids = tokenizer.encode(text, return_tensors="tf")
    embeddings = bangla_model(input_ids)[0]
    return embeddings



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [ ]:
max_sequence_length = 128  # You can adjust this based on your headlines
headline_texts = df['headline'].tolist()
headline_input = tokenizer(headline_texts, padding='max_length', truncation=True, max_length=max_sequence_length, return_tensors='tf')


# Preprocess numerical data
numerical_data = df['Close'].values.reshape(-1, 1)  # Assuming 'Close' is the numerical column
scaler = StandardScaler()
numerical_data_scaled = scaler.fit_transform(numerical_data)
# Reshape X_text to have the same dimensions as X_price
# X_text = np.expand_dims(X_text, axis=1)
# print(X_text[0].shape, X_text[1].shape)
# print(X_price)
# Split data into training and testing sets (now compatible dimensions)
# X_train, X_test, y_train, y_test = train_test_split(
#     np.concatenate([X_text, X_price], axis=1), y, test_size=0.2, random_state=42
# )
headline_train, headline_test, numerical_train, numerical_test, labels_train, labels_test = train_test_split(
    headline_input['input_ids'].numpy(), numerical_data_scaled, df['Close'].values, test_size=0.2, random_state=42
)

In [ ]:
# print(type(np.array(X_headline_encoded)))
# X_headline_encoded[0].numpy().shape
for i, t in enumerate(headline_input['input_ids']):
  print(f"{df['headline'].iloc[i]}\n{t.shape}")

In [ ]:
# Define Early Fusion model architecture with pre-encoded text tensors
text_input = Input(shape=(128,), name="text_input")  # Adjust shape based on embedding dimension
conv_layer = Reshape((128, 1))(text_input)  # Reshape to include channel dimension for Conv1D
conv_layer = Conv1D(filters=32, kernel_size=3, activation="relu")(conv_layer)
pool_layer = MaxPooling1D(pool_size=2)(conv_layer)

# Price processing branch
price_input = Input(shape=(1, ), name="price_input")  # Shape for single timestep
# Reshape the input tensor to include the timestep dimension
reshaped_price_input = Reshape((1, 1))(price_input)
lstm_layer = LSTM(32)(reshaped_price_input)

# Repeat the numerical input to match the sequence length of the text input
repeated_numerical_input = RepeatVector(64)(lstm_layer)  # Assuming sequence length after pooling is 64
padded_pool_layer = ZeroPadding1D(padding=(0, 1))(pool_layer)
# Fusion layer
merged = concatenate([padded_pool_layer, repeated_numerical_input])

# Output layer
output_layer = Dense(1, activation="linear")(merged)

# Create the model
model = Model(inputs=[text_input, price_input], outputs=output_layer)
model.compile(optimizer="adam", loss="mse")

# Print model summary
model.summary()


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text_input (InputLayer)     [(None, 128)]                0         []                            
                                                                                                  
 reshape_11 (Reshape)        (None, 128, 1)               0         ['text_input[0][0]']          
                                                                                                  
 price_input (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 conv1d_32 (Conv1D)          (None, 126, 32)              128       ['reshape_11[0][0]']          
                                                                                            

In [ ]:
# # Train the model with pre-encoded text tensors
# model.fit(
#     [X_headline_encoded, X_price], y_train, epochs=10, validation_data=([X_test_headline_encoded, X_test_price], y_test)
# )
print(labels_train)
# Train the model
model.fit([headline_train, numerical_train], labels_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
loss = model.evaluate([headline_test, numerical_test], labels_test)
print("Test Loss: ", loss)

# All Together

In [ ]:
def run_for(company_name):
  print("Running for: " + company_name)
  def merge_stock_and_news_data(company_name):
      company_data_path = f'drive/MyDrive/SoftComp/Dhaka Stock Exchange Historical Data/Adjusted Data/{company_name}.csv'
      news_data_path = 'drive/MyDrive/SoftComp/prothomalo_data.csv'

      # Read data from CSV files
      stock_df = pd.read_csv(company_data_path)
      news_df = pd.read_csv(news_data_path)

      # Columns to select from each DataFrame
      cols_from_stock = ['Date', 'Close']
      cols_from_news = ['Date', 'headline']

      # Convert 'Date' column to datetime format
      stock_df['Date'] = pd.to_datetime(stock_df['Date']).dt.strftime('%d-%m-%Y')
      news_df.rename(columns={'published_at': 'Date'}, inplace=True)

      # Merge DataFrames on 'Date' column
      merged_df = pd.merge(news_df[cols_from_news], stock_df[cols_from_stock], on='Date', how='inner')

      return merged_df

  # Example usage:
  # company_name = "ABBANK"

  df = merge_stock_and_news_data(company_name)
  # print(df.head())

  #====

  import tensorflow as tf
  from transformers import TFAutoModel, AutoTokenizer
  from transformers import TFBertModel, BertTokenizer

  # Load the tokenizer and model
  tokenizer = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
  bangla_model = TFBertModel.from_pretrained("sagorsarker/bangla-bert-base", from_pt=True)

  # Define a function to encode Bengali text
  def encode_bengali_text(text):
      input_ids = tokenizer.encode(text, return_tensors="tf")
      embeddings = bangla_model(input_ids)[0]
      return embeddings

  #==========

  max_sequence_length = 128  # You can adjust this based on your headlines
  headline_texts = df['headline'].tolist()
  headline_input = tokenizer(headline_texts, padding='max_length', truncation=True, max_length=max_sequence_length, return_tensors='tf')


  # Preprocess numerical data
  numerical_data = df['Close'].values.reshape(-1, 1)  # Assuming 'Close' is the numerical column
  scaler = StandardScaler()
  numerical_data_scaled = scaler.fit_transform(numerical_data)

  headline_train, headline_test, numerical_train, numerical_test, labels_train, labels_test = train_test_split(
      headline_input['input_ids'].numpy(), numerical_data_scaled, df['Close'].values, test_size=0.2, random_state=42
  )

  #==========

  # Define Early Fusion model architecture with pre-encoded text tensors
  text_input = Input(shape=(128,), name="text_input")  # Adjust shape based on embedding dimension
  conv_layer = Reshape((128, 1))(text_input)  # Reshape to include channel dimension for Conv1D
  conv_layer = Conv1D(filters=32, kernel_size=3, activation="relu")(conv_layer)
  pool_layer = MaxPooling1D(pool_size=2)(conv_layer)

  # Price processing branch
  price_input = Input(shape=(1, ), name="price_input")  # Shape for single timestep
  # Reshape the input tensor to include the timestep dimension
  reshaped_price_input = Reshape((1, 1))(price_input)
  lstm_layer = LSTM(32)(reshaped_price_input)

  # Repeat the numerical input to match the sequence length of the text input
  repeated_numerical_input = RepeatVector(64)(lstm_layer)  # Assuming sequence length after pooling is 64
  padded_pool_layer = ZeroPadding1D(padding=(0, 1))(pool_layer)
  # Fusion layer
  merged = concatenate([padded_pool_layer, repeated_numerical_input])

  # Output layer
  output_layer = Dense(1, activation="linear")(merged)

  # Create the model
  model = Model(inputs=[text_input, price_input], outputs=output_layer)
  model.compile(optimizer="adam", loss="mae")

  # Print model summary
  # model.summary()

  #========

  # Train the model
  model.fit([headline_train, numerical_train], labels_train, epochs=20, batch_size=32, validation_split=0.1)

  # Evaluate the model
  loss = model.evaluate([headline_test, numerical_test], labels_test)
  print("Test Loss: ", loss)
  return loss

In [ ]:
import os
import csv

# Directory path
directory = 'drive/MyDrive/SoftComp/Dhaka Stock Exchange Historical Data/Adjusted Data/'

# Get all files in the directory
files = os.listdir(directory)

# Filter CSV files and remove extension
csv_files = [file.split('.')[0] for file in files if file.endswith('.csv')]

losses = {}

# Simulating the process for the first 5 companies
for company in csv_files[:25]:
    loss = run_for(company)
    losses[company] = loss

# Save all losses to a single CSV file
output_file = os.path.join(directory, 'all_losses.csv')
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Company', 'Loss'])
    for company, loss in losses.items():
        writer.writerow([company, loss])

In [ ]:
 run_for("DACCADYE")

Running for: DACCADYE


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

Epoch 1/20
46/46 [==============================] - 4s 29ms/step - loss: 115.3068 - val_loss: 77.8491
Epoch 2/20
46/46 [==============================] - 1s 14ms/step - loss: 52.5552 - val_loss: 34.3366
Epoch 3/20
46/46 [==============================] - 1s 14ms/step - loss: 29.0709 - val_loss: 24.5080
Epoch 4/20
46/46 [==============================] - 1s 14ms/step - loss: 22.3818 - val_loss: 19.3363
Epoch 5/20
46/46 [==============================] - 1s 14ms/step - loss: 17.4533 - val_loss: 15.9837
Epoch 6/20
46/46 [==============================] - 0s 10ms/step - loss: 14.9522 - val_loss: 15.3328
Epoch 7/20
46/46 [==============================] - 0s 9ms/step - loss: 13.0775 - val_loss: 12.3362
Epoch 8/20
46/46 [==============================] - 0s 9ms/step - loss: 11.2982 - val_loss: 10.3471
Epoch 9/20
46/46 [==============================] - 0s 9ms/step - loss: 10.1044 - val_loss: 9.7984
Epoch 10/20
46/46 [==============================] - 0s 8ms/step - loss: 9.8605 - val_loss: 8.

7.690665245056152

|Company|Loss|
|---|---|
|COPPERTECH|3676\.3701171875|
|DACCADYE|697\.7158813476562|
|EMERALDOIL|1668\.7244873046875|
|EASTLAND|955\.0673217773438|
|DESCO|3441\.3203125|
|DBH|4648\.8525390625|
|ECABLES|16513\.33984375|
|CVOPRL|61835\.171875|
|CROWNCEMNT|22674\.333984375|
|CITYBANK|1034\.007080078125|
|DOMINAGE|1762\.115478515625|
|ENVOYTEX|1276\.8028564453125|
|FAMILYTEX|1653\.0316162109375|
|DELTASPINN|2299\.0712890625|
|BXSYNTH|1368\.7296142578125|
|CITYGENINS|1913\.43017578125|
|CONFIDCEM|5183\.931640625|
|FARCHEM|1099\.353515625|
|DHAKABANK|1139\.431884765625|
|BPPL|54990\.4921875|
|DHAKAINS|1439\.9873046875|
|DSHGARME|16711\.75390625|
|EBL|981\.372802734375|
|CENTRALINS|1460\.9859619140625|
|BSRMSTEEL|2805\.183837890625|